In [28]:
# Standard libraries
import sys
# Add your custom path
gems_tco_path = "/Users/joonwonlee/Documents/GEMS_TCO-1/src"
sys.path.append(gems_tco_path)

# Data manipulation and analysis
import pandas as pd
import numpy as np

from GEMS_TCO import kernels
from GEMS_TCO import data_preprocess 
from GEMS_TCO import kernels_new, kernels_reparam_space_time as kernels_reparam_space_time
from GEMS_TCO import orderings as _orderings 
from GEMS_TCO import load_data
from GEMS_TCO import alg_optimization, alg_opt_Encoder
from GEMS_TCO import configuration as config

from typing import Optional, List, Tuple
from pathlib import Path
from json import JSONEncoder

from GEMS_TCO.data_loader import load_data2
import torch
import torch.optim as optim
import time

Load monthly data

In [54]:
space: List[str] = ['1', '8']
lat_lon_resolution = [int(s) for s in space]
mm_cond_number: int = 20
years = ['2024']
month_range = [7] 

output_path = input_path = Path(config.mac_estimates_day_path)
data_load_instance = load_data2(config.mac_data_load_path)


df_map, ord_mm, nns_map = data_load_instance.load_maxmin_ordered_data_bymonthyear(
lat_lon_resolution=lat_lon_resolution, 
mm_cond_number=mm_cond_number,
years_=years, 
months_=month_range,
lat_range=[0.0, 5.0],      
lon_range=[123.0, 133.0] 
)

#days: List[str] = ['0', '31']
#days_s_e = [int(d) for d in days]
#days_list = list(range(days_s_e[0], days_s_e[1]))

Subsetting data to lat: [0.0, 5.0], lon: [123.0, 133.0]


Load daily data applying max-min ordering

In [55]:
daily_aggregated_tensors = [] 
daily_hourly_maps = []        

analysis_hour =2
k=0
for day_index in range(31):
  
    hour_start_index = day_index * 8 + k 


    hour_end_index = (day_index + 1) * 8 + k
    #hour_end_index = day_index*8 + 1
    #hour_end_index = day_index*8 + analysis_hour + k

    hour_indices = [hour_start_index, hour_end_index]
    
    # Load the data for the current day
    day_hourly_map, day_aggregated_tensor = data_load_instance.load_working_data(
        df_map, 
        hour_indices, 
        ord_mm=None,  
        dtype=torch.float 
    )
    # Append the day's data to their respective lists
    daily_aggregated_tensors.append(day_aggregated_tensor)
    daily_hourly_maps.append(day_hourly_map) 

print(daily_aggregated_tensors[0].shape)
#print(daily_hourly_maps[0])

nn = daily_aggregated_tensors[0].shape[0]

torch.Size([18240, 4])


Hyper parameters

In [56]:
v = 0.5 # smooth
mm_cond_number = 8
nheads = 300
#nheads = 1230
#lr = 0.01
#step = 80
#gamma_par = 0.5

# --- Placeholder Global Variables ---
# 💥 REVISED: Added lr, patience, factor. Removed step, gamma_par
lr=0.1
patience = 5       # Scheduler: Epochs to wait for improvement
factor = 0.5         # Scheduler: Factor to reduce LR by (e.g., 0.5 = 50% cut)
epochs=150

In [58]:
def cal(a):
    day_indices = [0] 
    for day_idx in day_indices:  

        daily_hourly_map = daily_hourly_maps[day_idx]
        daily_aggregated_tensor = daily_aggregated_tensors[day_idx]

        # Convert initial parameters to a list of 1-element tensors
        params_list = [
            torch.tensor([val], dtype=torch.float64, requires_grad=True, device=device_str) for val in a
        ]
        
        # 💡 CRITICAL: Concatenate the list into a single tensor
        # The full_likelihood function expects a single tensor, not a list
        params_tensor = torch.cat(params_list)

        # Assuming 'kernels_repar_space' has your 'fit_vecchia_adams' class
  
        model_instance = kernels_reparam_space_time.fit_vecchia_adams(
                smooth = v,
                input_map = daily_hourly_map,
                aggregated_data = daily_aggregated_tensor,
                nns_map = nns_map,
                mm_cond_number = mm_cond_number,
                nheads = nheads
            )
    
        # 💡 Pass the single 'params_tensor' and the correct 4-parameter spatial covariance function
        bb = model_instance.full_likelihood_avg(
            params = params_tensor, 
            input_data = daily_aggregated_tensor, 
            y = daily_aggregated_tensor[:,2], 
            covariance_function = model_instance.matern_cov_aniso_STABLE_log_reparam
        )
     
        cov_map = model_instance.cov_structure_saver(params_tensor, model_instance.matern_cov_aniso_STABLE_log_reparam)
        vecchia_nll = model_instance.vecchia_space_time_fullbatch( # Change this to your chosen Vecchia implementation
            params = params_tensor, 
            covariance_function = model_instance.matern_cov_aniso_STABLE_log_reparam, 
            cov_map = cov_map # Assuming cov_map is precomputed or computed internally
        )
 
    return bb, vecchia_nll

device_str = 'cuda' if torch.cuda.is_available() else 'cpu'
# adams and lfgs
#a= [3.2691297610712176, 0.6684659526552683, 0.4027674047362919, 0.8989261051578574, 6521.4909959990455]

#bb, nll_value = cal(a)
#print(f"Full Negative Log-Likelihood: {bb.item()*1120, nll_value.item()}")

def cc(n,nheads,mm):
    print(n**2 , nheads**2 + (n-nheads)*mm**3)
    return n**2 >nheads**2 + (n-nheads)*mm**3

cc(1000,300,8)

1000000 448400


True

# 8 hours 1120

init   Param 0: 3.0647
  Param 1: 0.3567
  Param 2: 0.1177
  Param 3: -4.6052
  Param 4: 0.0200
  Param 5: -0.0800
  Param 6: 0.4055

  to

--- Epoch 61 / Loss: 1.244809 ---
  Param 0: Value=3.9545, Grad=0.0025311473891214244
  Param 1: Value=1.1901, Grad=-0.0009350653303357313
  Param 2: Value=0.4366, Grad=0.0010677078901593694
  Param 3: Value=-3.6776, Grad=-0.00048638118522927384
  Param 4: Value=0.0279, Grad=-0.000609149651126906
  Param 5: Value=-0.1399, Grad=0.012523378927349654
  Param 6: Value=-0.0443, Grad=0.006097463420966644
  Max Abs Grad: 1.252338e-02





### 3rd vecchia 18126 adams

a = [4.2030, 1.6337, 0.4743, -3.7745, 0.0218, -0.1689, -1.3985]

    init_sigmasq   = 13.0754
    init_range_lat = 0.154
    init_range_lon = 0.1952
    init_nugget    = 0.2471
    init_beta      = 0.1513  # <-- NEW (Temporal range)
    init_advec_lat = 0.0218  # <-- NEW (Start with no advection)
    init_advec_lon = -0.1689  # <-- NEW (Start with no advection)

--- Epoch 11 / Loss: 1.242106 ---
  Param 0: Value=4.2030, Grad=0.00011921633020749384
  Param 1: Value=1.6337, Grad=2.0856591009061707e-07
  Param 2: Value=0.4743, Grad=8.995107359505467e-05
  Param 3: Value=-3.7745, Grad=-2.4886759378713545e-05
  Param 4: Value=0.0218, Grad=-0.00014679767411203022
  Param 5: Value=-0.1689, Grad=-9.612744279827595e-05
  Param 6: Value=-1.3985, Grad=0.00034032926084563323
  Max Abs Grad: 3.403293e-04


### 4th start from 3rd above

--- Epoch 1 / Loss: 1.242107 ---
  Param 0: Value=4.2045, Grad=0.0007321064034577786
  Param 1: Value=1.6337, Grad=-6.011924329571855e-05
  Param 2: Value=0.4741, Grad=0.00022945347336666866
  Param 3: Value=-3.7770, Grad=-2.528440912756315e-05
  Param 4: Value=0.0218, Grad=-0.00016348631302979882
  Param 5: Value=-0.1689, Grad=-8.778784781083261e-05
  Param 6: Value=-1.3980, Grad=0.0003882991902750959
  Max Abs Grad: 7.321064e-04
------------------------------
--- Epoch 11 / Loss: 1.242095 ---
  Param 0: Value=4.2037, Grad=-0.0021015039442420546
  Param 1: Value=1.6385, Grad=0.0001732094040773685
  Param 2: Value=0.4730, Grad=-0.0005118434541117669
  Param 3: Value=-3.7616, Grad=-6.805289675986991e-05
  Param 4: Value=0.0254, Grad=0.0015166640511121228
  Param 5: Value=-0.1641, Grad=0.0011050402880583106
  Param 6: Value=-1.4644, Grad=0.0001336375772895083
  Max Abs Grad: 2.101504e-03

  ------------------------------

  now init

    init_sigmasq   = 13.0031
    init_range_lat = 0.1533
    init_range_lon = 0.1942
    init_nugget    = 0.2312 
    init_range_time  = 1.2751  # <-- NEW (Temporal range)
    init_advec_lat = 0.0254  # <-- NEW (Start with no advection)
    init_advec_lon = -0.1641  # <-- NEW (Start with no advection)


In [21]:
# 1x8
print(nn)
a = [4.2030, 1.6337, 0.4743, -3.7745, 0.0218, -0.1689, -1.3985]
 # estimates
bb, nll_value = cal(a)
print(f"Full Negative Log-Likelihood: {bb.item()*nn, nll_value.item()*nn}")
a = [4.3030, 1.6337, 0.4743, -3.7745, 0.0218, -0.1689, -1.3985]
bb, nll_value = cal(a)
print(f"Full Negative Log-Likelihood: {bb.item()*nn, nll_value.item()*nn}")
a = [4.1030, 1.6337, 0.4743, -3.7745, 0.0218, -0.1689, -1.3985]
bb, nll_value = cal(a)
print(f"Full Negative Log-Likelihood: {bb.item()*nn, nll_value.item()*nn}")

18240
Full Negative Log-Likelihood: (24873.92914861954, 30091.34738593696)
Full Negative Log-Likelihood: (24951.323501001483, 30213.360204382538)
Full Negative Log-Likelihood: (24873.407190724567, 30047.750918891386)


In [27]:
# 8x1
print(nn)
a = [4.2030, 1.6337, 0.4743, -3.7745, 0.0218, -0.1689, -1.3985]
 # estimates
bb, nll_value = cal(a)
print(f"Full Negative Log-Likelihood: {bb.item()*nn, nll_value.item()*nn}")
a = [4.3030, 1.6337, 0.4743, -3.7745, 0.0218, -0.1689, -1.3985]
bb, nll_value = cal(a)
print(f"Full Negative Log-Likelihood: {bb.item()*nn, nll_value.item()*nn}")
a = [4.1030, 1.6337, 0.4743, -3.7745, 0.0218, -0.1689, -1.3985]
bb, nll_value = cal(a)
print(f"Full Negative Log-Likelihood: {bb.item()*nn, nll_value.item()*nn}")

17808
Full Negative Log-Likelihood: (25034.460446484678, 29081.903164642717)
Full Negative Log-Likelihood: (25073.0712016377, 29176.202387841746)
Full Negative Log-Likelihood: (25075.04504536982, 29066.054197441317)


In [41]:
# 4x2
print(nn)
a = [4.2030, 1.6337, 0.4743, -3.7745, 0.0218, -0.1689, -1.3985]
 # estimates
bb, nll_value = cal(a)
print(f"Full Negative Log-Likelihood: {bb.item()*nn, nll_value.item()*nn}")
a = [4.3030, 1.6337, 0.4743, -3.7745, 0.0218, -0.1689, -1.3985]
bb, nll_value = cal(a)
print(f"Full Negative Log-Likelihood: {bb.item()*nn, nll_value.item()*nn}")
a = [4.1030, 1.6337, 0.4743, -3.7745, 0.0218, -0.1689, -1.3985]
bb, nll_value = cal(a)
print(f"Full Negative Log-Likelihood: {bb.item()*nn, nll_value.item()*nn}")

17920
Full Negative Log-Likelihood: (27504.598234595054, 30013.098629078344)
Full Negative Log-Likelihood: (27581.091237640092, 30120.485263366845)
Full Negative Log-Likelihood: (27507.849833107794, 29984.444988568906)


In [46]:
# 2x4
print(nn)
a = [4.2030, 1.6337, 0.4743, -3.7745, 0.0218, -0.1689, -1.3985]
 # estimates
bb, nll_value = cal(a)
print(f"Full Negative Log-Likelihood: {bb.item()*nn, nll_value.item()*nn}")
a = [4.3030, 1.6337, 0.4743, -3.7745, 0.0218, -0.1689, -1.3985]
bb, nll_value = cal(a)
print(f"Full Negative Log-Likelihood: {bb.item()*nn, nll_value.item()*nn}")
a = [4.1030, 1.6337, 0.4743, -3.7745, 0.0218, -0.1689, -1.3985]
bb, nll_value = cal(a)
print(f"Full Negative Log-Likelihood: {bb.item()*nn, nll_value.item()*nn}")

18240
Full Negative Log-Likelihood: (27964.57050296843, 30646.21624063658)
Full Negative Log-Likelihood: (28028.578683142507, 30755.515702606983)
Full Negative Log-Likelihood: (27982.718061168925, 30617.132519016523)


###  vecchia 18126 adams 


a = [4.2035, 1.6284, 0.4787, -3.7756, 0.0208, -0.1665, -1.3916]

    init_sigmasq   = 13.134
    init_range_lat = 0.155
    init_range_lon = 0.196
    init_nugget    = 0.249
    init_beta      = 0.151  # <-- NEW (Temporal range)
    init_advec_lat = 0.0208  # <-- NEW (Start with no advection)
    init_advec_lon = -0.1665  # <-- NEW (Start with no advection)

In [27]:
# 4 x 4
print(nn)
a = [4.2035, 1.6284, 0.4787, -3.7756, 0.0208, -0.1665, -1.3916]
 # estimates
bb, nll_value = cal(a)
print(f"Full Negative Log-Likelihood: {bb.item()*nn, nll_value.item()*nn}")
a = [4.3035, 1.6284, 0.4787, -3.7756, 0.0208, -0.1665, -1.3916]
bb, nll_value = cal(a)
print(f"Full Negative Log-Likelihood: {bb.item()*nn, nll_value.item()*nn}")
a = [4.1035, 1.6284, 0.4787, -3.7756, 0.0208, -0.1665, -1.3916]
bb, nll_value = cal(a)
print(f"Full Negative Log-Likelihood: {bb.item()*nn, nll_value.item()*nn}")

8960
Full Negative Log-Likelihood: (14465.24800860151, 15086.920716483808)
Full Negative Log-Likelihood: (14512.760579502035, 15139.239578340457)
Full Negative Log-Likelihood: (14457.393090801805, 15074.151915743585)


In [12]:
# 4 x 2
print(nn)
a = [4.2035, 1.6284, 0.4787, -3.7756, 0.0208, -0.1665, -1.3916]
 # estimates
bb, nll_value = cal(a)
print(f"Full Negative Log-Likelihood: {bb.item()*nn, nll_value.item()*nn}")
a = [4.3035, 1.6284, 0.4787, -3.7756, 0.0208, -0.1665, -1.3916]
bb, nll_value = cal(a)
print(f"Full Negative Log-Likelihood: {bb.item()*nn, nll_value.item()*nn}")
a = [4.1035, 1.6284, 0.4787, -3.7756, 0.0208, -0.1665, -1.3916]
bb, nll_value = cal(a)
print(f"Full Negative Log-Likelihood: {bb.item()*nn, nll_value.item()*nn}")


17920
Full Negative Log-Likelihood: (27507.89072775894, 30017.214685763123)
Full Negative Log-Likelihood: (27585.868515521863, 30126.522617552284)
Full Negative Log-Likelihood: (27509.47018195187, 29986.413822608047)


In [6]:
# 8 x 1
print(nn)
a = [4.2035, 1.6284, 0.4787, -3.7756, 0.0208, -0.1665, -1.3916]
 # estimates
bb, nll_value = cal(a)
print(f"Full Negative Log-Likelihood: {bb.item()*nn, nll_value.item()*nn}")
a = [4.3035, 1.6284, 0.4787, -3.7756, 0.0208, -0.1665, -1.3916]
bb, nll_value = cal(a)
print(f"Full Negative Log-Likelihood: {bb.item()*nn, nll_value.item()*nn}")
a = [4.1035, 1.6284, 0.4787, -3.7756, 0.0208, -0.1665, -1.3916]
bb, nll_value = cal(a)
print(f"Full Negative Log-Likelihood: {bb.item()*nn, nll_value.item()*nn}")

17808
Full Negative Log-Likelihood: (25035.58241568927, 29087.279192051596)
Full Negative Log-Likelihood: (25075.405252340654, 29183.24564612692)
Full Negative Log-Likelihood: (25074.780483443268, 29069.55974577143)


In [16]:
# 1x8
print(nn)
a = [4.2035, 1.6284, 0.4787, -3.7756, 0.0208, -0.1665, -1.3916]
 # estimates
bb, nll_value = cal(a)
print(f"Full Negative Log-Likelihood: {bb.item()*nn, nll_value.item()*nn}")
a = [4.3035, 1.6284, 0.4787, -3.7756, 0.0208, -0.1665, -1.3916]
bb, nll_value = cal(a)
print(f"Full Negative Log-Likelihood: {bb.item()*nn, nll_value.item()*nn}")
a = [4.1035, 1.6284, 0.4787, -3.7756, 0.0208, -0.1665, -1.3916]
bb, nll_value = cal(a)
print(f"Full Negative Log-Likelihood: {bb.item()*nn, nll_value.item()*nn}")

18240
Full Negative Log-Likelihood: (24874.534970387504, 30094.38446997377)
Full Negative Log-Likelihood: (24954.27221989919, 30218.563846812976)
Full Negative Log-Likelihood: (24871.415197699913, 30048.373555436625)


In [20]:
# 2 x 4
print(nn)
a = [4.2035, 1.6284, 0.4787, -3.7756, 0.0208, -0.1665, -1.3916]
 # estimates
bb, nll_value = cal(a)
print(f"Full Negative Log-Likelihood: {bb.item()*nn, nll_value.item()*nn}")
a = [4.3035, 1.6284, 0.4787, -3.7756, 0.0208, -0.1665, -1.3916]
bb, nll_value = cal(a)
print(f"Full Negative Log-Likelihood: {bb.item()*nn, nll_value.item()*nn}")
a = [4.1035, 1.6284, 0.4787, -3.7756, 0.0208, -0.1665, -1.3916]
bb, nll_value = cal(a)
print(f"Full Negative Log-Likelihood: {bb.item()*nn, nll_value.item()*nn}")


18240
Full Negative Log-Likelihood: (27964.897506272282, 30649.958282792086)
Full Negative Log-Likelihood: (28031.76517041291, 30761.4342459134)
Full Negative Log-Likelihood: (27979.882378669332, 30618.449533611387)


# Adams vecchia 4560

In [ ]:
# 4 x 2

a = [3.3671047020222225, 0.8156127304020526, 0.44950475169697685, -3.4258506297302818, 0.03545588816692566, -0.21467686574898073, 1.0496833323567616]
 # estimates
bb, nll_value = cal(a)
print(f"Full Negative Log-Likelihood: {bb.item()*nn, nll_value.item()*nn}")
a = [3.4671047020222225, 0.8156127304020526, 0.44950475169697685, -3.4258506297302818, 0.03545588816692566, -0.21467686574898073, 1.0496833323567616]
bb, nll_value = cal(a)
print(f"Full Negative Log-Likelihood: {bb.item()*nn, nll_value.item()*nn}")
a = [3.2671047020222225, 0.8156127304020526, 0.44950475169697685, -3.4258506297302818, 0.03545588816692566, -0.21467686574898073, 1.0496833323567616]
bb, nll_value = cal(a)
print(f"Full Negative Log-Likelihood: {bb.item()*nn, nll_value.item()*nn}")


Full Negative Log-Likelihood: (27284.87891690571, 30230.921299950904)
Full Negative Log-Likelihood: (27287.398135874384, 30312.951119572594)
Full Negative Log-Likelihood: (27314.201956871697, 30190.681810740316)


In [21]:
# 2 x 4

a = [3.3671047020222225, 0.8156127304020526, 0.44950475169697685, -3.4258506297302818, 0.03545588816692566, -0.21467686574898073, 1.0496833323567616]
 # estimates
bb, nll_value = cal(a)
print(f"Full Negative Log-Likelihood: {bb.item()*nn, nll_value.item()*nn}")
a = [3.4671047020222225, 0.8156127304020526, 0.44950475169697685, -3.4258506297302818, 0.03545588816692566, -0.21467686574898073, 1.0496833323567616]
bb, nll_value = cal(a)
print(f"Full Negative Log-Likelihood: {bb.item()*nn, nll_value.item()*nn}")
a = [3.2671047020222225, 0.8156127304020526, 0.44950475169697685, -3.4258506297302818, 0.03545588816692566, -0.21467686574898073, 1.0496833323567616]
bb, nll_value = cal(a)
print(f"Full Negative Log-Likelihood: {bb.item()*nn, nll_value.item()*nn}")


Full Negative Log-Likelihood: (27760.86861995639, 30863.85563544486)
Full Negative Log-Likelihood: (27758.090711892113, 30951.85445601243)
Full Negative Log-Likelihood: (27795.136196782616, 30818.79219485778)


In [7]:
# 8 x 1

a = [3.3671047020222225, 0.8156127304020526, 0.44950475169697685, -3.4258506297302818, 0.03545588816692566, -0.21467686574898073, 1.0496833323567616]
 # estimates
bb, nll_value = cal(a)
print(f"Full Negative Log-Likelihood: {bb.item()*nn, nll_value.item()*nn}")
a = [3.4671047020222225, 0.8156127304020526, 0.44950475169697685, -3.4258506297302818, 0.03545588816692566, -0.21467686574898073, 1.0496833323567616]
bb, nll_value = cal(a)
print(f"Full Negative Log-Likelihood: {bb.item()*nn, nll_value.item()*nn}")
a = [3.2671047020222225, 0.8156127304020526, 0.44950475169697685, -3.4258506297302818, 0.03545588816692566, -0.21467686574898073, 1.0496833323567616]
bb, nll_value = cal(a)
print(f"Full Negative Log-Likelihood: {bb.item()*nn, nll_value.item()*nn}")

Full Negative Log-Likelihood: (25112.819569132364, 29380.73705549491)
Full Negative Log-Likelihood: (25121.97074321511, 29447.122691895336)
Full Negative Log-Likelihood: (25131.81761737561, 29353.245663447473)


In [17]:
# 1 x 8 

a = [3.3671047020222225, 0.8156127304020526, 0.44950475169697685, -3.4258506297302818, 0.03545588816692566, -0.21467686574898073, 1.0496833323567616]
 # estimates
bb, nll_value = cal(a)
print(f"Full Negative Log-Likelihood: {bb.item()*nn, nll_value.item()*nn}")
a = [3.4671047020222225, 0.8156127304020526, 0.44950475169697685, -3.4258506297302818, 0.03545588816692566, -0.21467686574898073, 1.0496833323567616]
bb, nll_value = cal(a)
print(f"Full Negative Log-Likelihood: {bb.item()*nn, nll_value.item()*nn}")
a = [3.2671047020222225, 0.8156127304020526, 0.44950475169697685, -3.4258506297302818, 0.03545588816692566, -0.21467686574898073, 1.0496833323567616]
bb, nll_value = cal(a)
print(f"Full Negative Log-Likelihood: {bb.item()*nn, nll_value.item()*nn}")

Full Negative Log-Likelihood: (25216.564907044954, 30376.987048091472)
Full Negative Log-Likelihood: (25236.289439198958, 30478.309911251905)
Full Negative Log-Likelihood: (25225.142959741937, 30317.8530983233)


# l bfgs 18126

init:     init_sigmasq   = 15.0
    init_range_lat = 0.66 
    init_range_lon = 0.7 
    init_nugget    = 1.5
    init_beta      = 0.1
    init_advec_lat = 0.02
    init_advec_lon = -0.08
    

  Raw (vecc) Parameters: [4.286645422792349, 1.7396404064146764, 0.48907270259813174, -3.7770937316117275, 0.02048493270070447, -0.16411459496379438, -12.055733094871497]
  Interpretable Parameters:
    sigma_sq  : 12.768804
    range_lon : 0.175584
    range_lat : 0.137494
    beta      : 0.151291
    advec_lat : 0.020485
    advec_lon : -0.164115
    nugget    : 0.000006


    init_sigmasq   = 13.059
    init_range_lat = 0.154 
    init_range_lon = 0.195 
    init_nugget    = 0.247
    init_range_time = 1.283     
    init_advec_lat = 0.0218
    init_advec_lon = -0.1689


    

In [ ]:

# 4 x 2 
a = [4.286645422792349, 1.7396404064146764, 0.48907270259813174, -3.7770937316117275, 0.02048493270070447, -0.16411459496379438, -12.055733094871497]
 # estimates
bb, nll_value = cal(a)
print(f"Full Negative Log-Likelihood: {bb.item()*nn, nll_value.item()*nn}")

a = [4.386645422792349, 1.7396404064146764, 0.48907270259813174, -3.7770937316117275, 0.02048493270070447, -0.16411459496379438, -12.055733094871497]
bb, nll_value = cal(a)

print(f"Full Negative Log-Likelihood: {bb.item()*nn, nll_value.item()*nn}")

a = [4.186645422792349, 1.7396404064146764, 0.48907270259813174, -3.7770937316117275, 0.02048493270070447, -0.16411459496379438, -12.055733094871497]
bb, nll_value = cal(a)

print(f"Full Negative Log-Likelihood: {bb.item()*nn, nll_value.item()*nn}")


Full Negative Log-Likelihood: (27571.224463908584, 30006.250541168938)
Full Negative Log-Likelihood: (27656.8858775873, 30102.507800218038)
Full Negative Log-Likelihood: (27570.786928251575, 29994.10285530667)


In [22]:
# 2 x 4 
a = [4.286645422792349, 1.7396404064146764, 0.48907270259813174, -3.7770937316117275, 0.02048493270070447, -0.16411459496379438, -12.055733094871497]
 # estimates
bb, nll_value = cal(a)
print(f"Full Negative Log-Likelihood: {bb.item()*nn, nll_value.item()*nn}")

a = [4.386645422792349, 1.7396404064146764, 0.48907270259813174, -3.7770937316117275, 0.02048493270070447, -0.16411459496379438, -12.055733094871497]
bb, nll_value = cal(a)

print(f"Full Negative Log-Likelihood: {bb.item()*nn, nll_value.item()*nn}")

a = [4.186645422792349, 1.7396404064146764, 0.48907270259813174, -3.7770937316117275, 0.02048493270070447, -0.16411459496379438, -12.055733094871497]
bb, nll_value = cal(a)

print(f"Full Negative Log-Likelihood: {bb.item()*nn, nll_value.item()*nn}")

Full Negative Log-Likelihood: (28025.017555959843, 30641.453435507872)
Full Negative Log-Likelihood: (28100.4588919504, 30738.41864415426)
Full Negative Log-Likelihood: (28037.557672767478, 30630.206079023625)


In [8]:
# 8 x 1 
a = [4.286645422792349, 1.7396404064146764, 0.48907270259813174, -3.7770937316117275, 0.02048493270070447, -0.16411459496379438, -12.055733094871497]
 # estimates
bb, nll_value = cal(a)
print(f"Full Negative Log-Likelihood: {bb.item()*nn, nll_value.item()*nn}")

a = [4.386645422792349, 1.7396404064146764, 0.48907270259813174, -3.7770937316117275, 0.02048493270070447, -0.16411459496379438, -12.055733094871497]
bb, nll_value = cal(a)

print(f"Full Negative Log-Likelihood: {bb.item()*nn, nll_value.item()*nn}")

a = [4.186645422792349, 1.7396404064146764, 0.48907270259813174, -3.7770937316117275, 0.02048493270070447, -0.16411459496379438, -12.055733094871497]
bb, nll_value = cal(a)

print(f"Full Negative Log-Likelihood: {bb.item()*nn, nll_value.item()*nn}")

Full Negative Log-Likelihood: (25087.173877925437, 29057.76265047305)
Full Negative Log-Likelihood: (25125.52526227867, 29144.4408487824)
Full Negative Log-Likelihood: (25138.43292513302, 29055.61247671195)


In [18]:
# 1x8 
a = [4.286645422792349, 1.7396404064146764, 0.48907270259813174, -3.7770937316117275, 0.02048493270070447, -0.16411459496379438, -12.055733094871497]
 # estimates
bb, nll_value = cal(a)
print(f"Full Negative Log-Likelihood: {bb.item()*nn, nll_value.item()*nn}")

a = [4.386645422792349, 1.7396404064146764, 0.48907270259813174, -3.7770937316117275, 0.02048493270070447, -0.16411459496379438, -12.055733094871497]
bb, nll_value = cal(a)

print(f"Full Negative Log-Likelihood: {bb.item()*nn, nll_value.item()*nn}")

a = [4.186645422792349, 1.7396404064146764, 0.48907270259813174, -3.7770937316117275, 0.02048493270070447, -0.16411459496379438, -12.055733094871497]
bb, nll_value = cal(a)

print(f"Full Negative Log-Likelihood: {bb.item()*nn, nll_value.item()*nn}")

Full Negative Log-Likelihood: (24922.442143139866, 30084.56340991114)
Full Negative Log-Likelihood: (25002.097720943595, 30193.254477814524)
Full Negative Log-Likelihood: (24930.324672364448, 30060.356959199904)


## second lfbgs
a = [4.2030, 1.6337, 0.4743, -3.7745, 0.0218, -0.1689, -1.3985]
init from 18126 adams vecchia

    init_sigmasq   = 13.059
    init_range_lat = 0.154 
    init_range_lon = 0.195 
    init_nugget    = 0.247
    init_range_time = 1.283     
    init_advec_lat = 0.0218
    init_advec_lon = -0.1689



FINAL STATE: Step 1, Loss: 1.2421059448955856
  Raw (vecc) Parameters: [4.286642426458099, 1.7396166546213205, 0.48908102342543236, -3.777078889337886, 0.020486826416164613, -0.16410986090084237, -11.813252497216158]
  Interpretable Parameters:
    sigma_sq  : 12.769069
    range_lon : 0.175588
    range_lat : 0.137496
    range_time = 1.160   
    advec_lat : 0.020487
    advec_lon : -0.164110
    nugget    : 0.000007

result likelihood almost same up to 1


### dw adams lbfgs

Final Parameters (Natural Scale): sigmasq: 11.7969, range_lat: 0.1104, range_lon: 0.1643, beta: 0.1864, advec_lat: 0.0223, advec_lon: -0.1672, nugget: 0.0000

a = [4.2739, 1.8060, 0.7948, -3.3599, 0.0223, -0.1672, -11.8381]

init:

    init_sigmasq   = 15.0
    init_range_lat = 0.66 
    init_range_lon = 0.7 
    init_nugget    = 1.5
    init_range_time = 1.28 # Temporal range ratio
    init_advec_lat = 0.02
    init_advec_lon = -0.08

In [ ]:
#4x2

a = [4.2739, 1.8060, 0.7948, -3.3599, 0.0223, -0.1672, -11.8381]
 # estimates
bb, nll_value = cal(a)
print(f"Full Negative Log-Likelihood: {bb.item()*nn, nll_value.item()*nn}")

a = [4.3739, 1.8060, 0.7948, -3.3599, 0.0223, -0.1672, -11.8381]
bb, nll_value = cal(a)

print(f"Full Negative Log-Likelihood: {bb.item()*nn, nll_value.item()*nn}")

a =[4.1739, 1.8060, 0.7948, -3.3599, 0.0223, -0.1672, -11.8381]
bb, nll_value = cal(a)

print(f"Full Negative Log-Likelihood: {bb.item()*nn, nll_value.item()*nn}")


Full Negative Log-Likelihood: (27663.61643365846, 30038.05509451633)
Full Negative Log-Likelihood: (27751.761739119647, 30099.26997706008)
Full Negative Log-Likelihood: (27660.43374771087, 30064.63519675999)


In [23]:
# 2x4

a = [4.2739, 1.8060, 0.7948, -3.3599, 0.0223, -0.1672, -11.8381]
 # estimates
bb, nll_value = cal(a)
print(f"Full Negative Log-Likelihood: {bb.item()*nn, nll_value.item()*nn}")

a = [4.3739, 1.8060, 0.7948, -3.3599, 0.0223, -0.1672, -11.8381]
bb, nll_value = cal(a)

print(f"Full Negative Log-Likelihood: {bb.item()*nn, nll_value.item()*nn}")

a =[4.1739, 1.8060, 0.7948, -3.3599, 0.0223, -0.1672, -11.8381]
bb, nll_value = cal(a)

print(f"Full Negative Log-Likelihood: {bb.item()*nn, nll_value.item()*nn}")

Full Negative Log-Likelihood: (28148.882860233818, 30694.53034508418)
Full Negative Log-Likelihood: (28253.351520518783, 30756.26207440437)
Full Negative Log-Likelihood: (28129.342821632996, 30722.221980320126)


In [9]:
# 8x1

a = [4.2739, 1.8060, 0.7948, -3.3599, 0.0223, -0.1672, -11.8381]
 # estimates
bb, nll_value = cal(a)
print(f"Full Negative Log-Likelihood: {bb.item()*nn, nll_value.item()*nn}")

a = [4.3739, 1.8060, 0.7948, -3.3599, 0.0223, -0.1672, -11.8381]
bb, nll_value = cal(a)

print(f"Full Negative Log-Likelihood: {bb.item()*nn, nll_value.item()*nn}")

a =[4.1739, 1.8060, 0.7948, -3.3599, 0.0223, -0.1672, -11.8381]
bb, nll_value = cal(a)

print(f"Full Negative Log-Likelihood: {bb.item()*nn, nll_value.item()*nn}")

Full Negative Log-Likelihood: (25132.72413697951, 29025.28064471326)
Full Negative Log-Likelihood: (25163.403949031483, 29086.210483464387)
Full Negative Log-Likelihood: (25192.461515354047, 29051.58677584674)


In [19]:
#1x8

a = [4.2739, 1.8060, 0.7948, -3.3599, 0.0223, -0.1672, -11.8381]
 # estimates
bb, nll_value = cal(a)
print(f"Full Negative Log-Likelihood: {bb.item()*nn, nll_value.item()*nn}")

a = [4.3739, 1.8060, 0.7948, -3.3599, 0.0223, -0.1672, -11.8381]
bb, nll_value = cal(a)

print(f"Full Negative Log-Likelihood: {bb.item()*nn, nll_value.item()*nn}")

a =[4.1739, 1.8060, 0.7948, -3.3599, 0.0223, -0.1672, -11.8381]
bb, nll_value = cal(a)

print(f"Full Negative Log-Likelihood: {bb.item()*nn, nll_value.item()*nn}")

Full Negative Log-Likelihood: (25127.533795742995, 30156.93720965503)
Full Negative Log-Likelihood: (25255.617221069962, 30232.719974261672)
Full Negative Log-Likelihood: (25081.895293844787, 30169.100036681968)


In [ ]:
0.152**2

In [64]:
0.195/0.152

1.2828947368421053

In [ ]:
# second dw l bfgs

## init
init_sigmasq   = 14.360
init_range_lat = 0.245
init_range_lon = 0.304
init_nugget    = 0.957
init_range_time = 1.28 # <-- NEW (Temporal range)
init_advec_lat = 0.0279  # <-- NEW (Start with no advection)
init_advec_lon = -0.1399  # <-- NEW (Start with no advection)

almost same result except nugget
estimates

a = [4.2739, 1.8060, 0.7948, -3.3599, 0.0223, -0.1672, -11.8381]


# third dw using adams using dw lfgs estimate as init
a = [4.2739, 1.8060, 0.7948, -3.3599, 0.0223, -0.1672, -11.8381]

init_sigmasq   = 11.7969
init_range_lat = 0.1104 
init_range_lon = 0.1643 
init_nugget    = 0.0001
init_range_time = 1.28 # Temporal range ratio
init_advec_lat = 0.0223
init_advec_lon = -0.1672

still almost same result
 Loss: 36210.0742 | Max Grad: 9.033203e-03
  Params (Raw Log): log_phi1: 4.2738, log_phi2: 1.8060, log_phi3: 0.7948, log_phi4: -3.3598, advec_lat: 0.0223, advec_lon: -0.1672, log_nugget: -9.3075
